In [2]:
import pandas as pd
from gensim import corpora, models
from gensim.models import Phrases

df = pd.read_csv("January_stories_with_date.csv", index_col=0)

In [3]:
lemmas_df = df[df["lemmas"].apply(lambda x: len(eval(x))>=50)] # Getting rid of stories with less than 50 lemmas
print(len(df))
print(len(lemmas_df))

42262
31791


In [10]:
# dates = len(df)*["2016-01-01"]
# df['date'] = dates
#df['date'] = pd.to_datetime(df['date'])
lemmas_df['date'] = pd.to_datetime(lemmas_df['date'])
# df.loc[df.index.isin(range(41746, 42261)), 'date'] = '2016-01-31'
#df.to_csv("january_stories_with_date.csv")
#df.loc[42261]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
#jan1_jan3_df = lemmas_df[lemmas_df["date"].isin(pd.date_range("2016-01-01", "2016-01-03"))]
#len(jan1_jan3_df)


In [ ]:
# jan1_jan3_lemmas = jan1_jan3_df["lemmas"].tolist()
# jan1_jan3_new_lemmas = [eval(i) for i in jan1_jan3_lemmas]


In [ ]:
# jan1_jan3_dictionary = corpora.Dictionary(jan1_jan3_new_lemmas)
# #len(new_dictionary)
# corpus = [jan1_jan3_dictionary.doc2bow(story) for story in jan1_jan3_new_lemmas]
#len(corpus)

In [17]:
def windowLdaModel(day1, day3):
    """
    This function takes two inputs, start date and end date, then it builds an lda model for those days' data and concatenate the lda dataframe with the given
    story dataframe
    """
    day1_day3_df = lemmas_df[lemmas_df["date"].isin(pd.date_range("2016-01-%s" % (day1), "2016-01-%s" % (day3)))]
    day1_day3_lemmas = day1_day3_df["lemmas"].tolist()
    day1_day3_new_lemmas = [eval(i) for i in day1_day3_lemmas]
    day1_day3_dictionary = corpora.Dictionary(day1_day3_new_lemmas)
    print("Dictionary Length: %d" % len((day1_day3_dictionary)))
    corpus = [day1_day3_dictionary.doc2bow(story) for story in day1_day3_new_lemmas]
    print("Corpus Length: %d" % (len(corpus)))

    tfidf_model = models.TfidfModel(corpus)  # create tf.idf model
    # tfidf_model.save("tfidf_model")
    tfidf_corpus = tfidf_model[corpus]
    num_topics = 100
    lda_model = models.ldamodel.LdaModel(corpus=tfidf_corpus, id2word=day1_day3_dictionary, num_topics=num_topics, 
                                         update_every=0, chunksize=5000, passes=20)
    # lda_model.save("lda_model")
    lda_corpus = lda_model[tfidf_corpus]
    # corpora.MmCorpus.serialize("lda_corpus.mm", lda_corpus)
    topics_found_lda = lda_model.print_topics(num_topics=5, num_words=10)
    all_topics = lda_model.print_topics(num_topics=100, num_words=10)

    lda_df = pd.DataFrame(columns=range(100))
    
    for i in range(len(day1_day3_new_lemmas)):
        doc = lda_corpus[i]
        for top, prob in doc:
            lda_df.set_value(i, top, prob)
    print(doc)

    new_day1_day3_df = pd.concat([day1_day3_df, lda_df], axis=1, join='inner')
    new_day1_day3_df.to_csv("%s_%s.csv" %(day1, day3))
    
    return (new_day1_day3_df, all_topics)


In [12]:
jan1_jan3_lda_df = windowLdaModel(1, 3)

Dictionary Length: 28846
Corpus Length: 2283
[(24, 0.022070508020536068), (26, 0.35224413451439729), (55, 0.38058914061868837), (96, 0.069598951857333155), (99, 0.085032780497353883)]


In [16]:
%time jan2_jan4_lda_df = windowLdaModel(2, 4)

Dictionary Length: 26092
Corpus Length: 2144
[(18, 0.038579439317452897), (43, 0.012446499166766378), (46, 0.046806055292100905), (55, 0.03027218610996887), (73, 0.016912500782063786), (80, 0.077542283754546484), (88, 0.35430518497609176), (95, 0.33997115639085662)]
CPU times: user 11min 17s, sys: 9.45 s, total: 11min 26s
Wall time: 11min 45s


In [18]:
%time jan3_jan5_lda_df = windowLdaModel(3, 5)
jan3_jan5_df, jan3_jan5_topics = jan3_jan5_lda_df

Dictionary Length: 27966
Corpus Length: 2827
[(21, 0.22936348283749131), (74, 0.59545177843313124)]
CPU times: user 17min 37s, sys: 15.7 s, total: 17min 53s
Wall time: 18min 38s


In [32]:
%time jan4_jan6_lda_df = windowLdaModel(4, 6)
jan4_jan6_df, jan4_jan6_topics = jan4_jan6_lda_df

Dictionary Length: 31280
Corpus Length: 3582
[(1, 0.018051042448580856), (17, 0.025908111034569501), (22, 0.012327293027074337), (49, 0.06450956090420594), (69, 0.046378030036298375), (73, 0.030930375547544433), (89, 0.12793089829343171), (90, 0.49497928960136406), (95, 0.068710911851646933)]
CPU times: user 23min 22s, sys: 23.3 s, total: 23min 45s
Wall time: 25min 18s


In [34]:
%time jan5_jan7_lda_df = windowLdaModel(5, 7)
jan5_jan7_df, jan5_jan7_topics = jan5_jan7_lda_df

Dictionary Length: 33374
Corpus Length: 3857
[(14, 0.11856987739938979), (40, 0.0589533709641527), (43, 0.013774361082759879), (46, 0.06368397702390001), (80, 0.023838361108808749), (93, 0.64641756099623493)]
CPU times: user 23min 10s, sys: 18.3 s, total: 23min 28s
Wall time: 23min 46s


In [ ]:
%time jan6_jan8_lda_df = windowLdaModel(6, 8)
jan6_jan8_df, jan6_jan8_topics = jan6_jan8_lda_df

Dictionary Length: 34191
Corpus Length: 3822


In [ ]:
#new_jan1_jan3
